# Solving the model

## Initial setup
- Import libraries
- Specify the GMWO API base URL and your access token

In [1]:
import requests

base_url = 'https://model.oxfordeconomics.com/api'
access_token = <See https://model.oxfordeconomics.com/api/docs/#authentication>
headers = {'Authorization' : f'Bearer {access_token}'}
releasesFolder = "oxford-economics/releases/gem"
    

## Find an input forecast
To solve the model, you'll need an input forecast. You'll need to know the full path to the forecast in GMWO. E.g. `"/oxford-economics/releases/GEM/Oct23_1 25yr"`.

In this example, we use the `/resources` endpoint to find the path to the latest GEM release.

In [2]:
gemFolderResponse = requests.get(f"{base_url}/v1/resources/{releasesFolder}", headers=headers)
gemFolder = gemFolderResponse.json()

# Get forecasts only (i.e. filter out subdirectories and other file types)
gemForecasts = filter(lambda item: (item["Type"] == 'Forecast'), gemFolder["Children"])

# Sort by latest version of each forecast
gemForecastsLatestFirst = sorted(gemForecasts, key=lambda forecast: (forecast["Versions"][-1]["CreatedAt"]), reverse=True)
sourceForecast = gemForecastsLatestFirst[0]
sourceForecast

{'Type': 'Forecast',
 'EconomicDomain': 'MACRO',
 'ForecastUrl': '/v1/forecasts/=2198fad7-c646-43f7-b81c-f86dd4eea113',
 'Versions': [{'Name': 'Oct23_1 25yr',
   'EconomicDomain': 'MACRO',
   'Range': {'From': '1980Q1', 'To': '2050Q4'},
   'Id': 'd3203eab-fb65-41dc-83d9-1fb87e1c3158',
   'Version': 0,
   'CreatedAt': '2023-10-11T18:55:49.3903852+00:00'}],
 'ResourceType': 'Forecast',
 'Id': '2198fad7-c646-43f7-b81c-f86dd4eea113',
 'Name': 'Oct23_1 25yr',
 'Path': '/oxford-economics/releases/GEM/Oct23_1 25yr',
 'Archiving': None,
 'Product': {'TypeCode': 'OEF', 'Code': 'GBLMACRLM25_ONLINE'}}

## Upload model changes (optional)
Typically, you'll be making changes to the forecast before solving. These changes can be either:
- a set of model commands in *3FS* syntax
- an xlsx workbook containing imposed values for one or more variables, see example format [`here`](https://my.oxfordeconomics.com/reportaction/DF98456085CB405489340D/Toc)

If you have a set of changes in a local file, you'll need to upload them to the GMWO file system before you can use them in a model solution. In this example, we're uploading an import workbook.

In [3]:
importFileLocal = "C:\\Path\\To\\My-Changes.xlsx"
importFileGmwo = "me/My-Changes.xlsx"
multipart_form_data = {
    'file': ('data.xlsx', open(importFileLocal, 'rb')), # The name 'data.xlsx' here can be anything 
    'FileExtension': (None, 'xlsx') # You must provide the 'FileExtension' form value containing the file extension without a dot
}

uploadResponse = requests.post(f"{base_url}/v1/files/upload/{importFileGmwo}", files=multipart_form_data, headers=headers)
uploadedFile = uploadResponse.json()
uploadedFile

{'Type': 'GenericFile',
 'MimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
 'FileType': 'Xlsx',
 'Versions': [{'Id': 'cac9499e-99f3-4e51-b776-84c87b89c147',
   'Version': 0,
   'CreatedAt': '2023-10-16T10:16:22.656079+00:00'}],
 'ResourceType': 'OtherFile',
 'Id': '5926e3ad-a736-40ef-9cdb-9f02833af104',
 'Name': 'data.xlsx',
 'Path': '/tim-stott/data.xlsx',
 'Archiving': None,
 'Product': None}

## Trigger the model solution
Send a request to initiate the solution. The response will be a "Queued" operation. You must send further requests to wait for this operation to complete.

**Note**: requests to enqueue an operation such as this are subject to stricter rate limits. Please refer to the [`Throttling`](https://model.oxfordeconomics.com/api/docs/#throttling-request-limits-) section in the API guide.

In [4]:
# The solution request body expects JSON of the form below.

solveRequest = {
    # Path to the input forecast. If you already know this, then replace with e.g. "oxford-economics/releases/GEM/Oct23_1 25yr"
    "InputForecast": sourceForecast["Path"],

    # Range of periods to be solved
    "SolutionRange": {
        "From": "2023Q2",
        "To": "2027Q4"
    },

    ## Inline model commands to be applied before solving
    ## Cannot be used with "CommandsFile"
    ## Omit this and "CommandsFile" to solve the model without changes
    # "Commands": "MULTIPLY C:US 2023Q2 1.01, 1.01, 1.02\nFIX C:US 2023Q2:2028Q4",

    ## Path to file containing model commands or valid import data to be applied before solving
    ## Cannot be used with "Commands"
    ## Omit this and "Commands" to solve the model without changes
    # "CommandsFile": "/me/path/to/commands.3fs",
    "CommandsFile": uploadedFile["Path"],

    ## Optionally restrict solution to certain model groups or locations
    # "Groups": [
    #    "US",
    #    "CHINA"
    # ],

    ## Optional name for the solution operation. This name will appear in API requests
    ## for this operation and in the 'History' panel of the Run Model user interface. It's also 
    ## used to name the other artifacts generated during the operation (besides the output forecast, see below)
    # "OperationName": "Example name",

    # Path at which to save the newly-solved forecast
    "OutputForecast": "/me/path/to/new-forecast"
    
}
solveOperationResponse = requests.post(f"{base_url}/v1/operations/solve", headers=headers, json=solveRequest)
solveOperation = solveOperationResponse.json()
solveOperation


{'Artifacts': [],
 'Resources': [],
 'Id': '23613',
 'CreatedAt': '2023-10-16T10:18:42.423+00:00',
 'StartedAt': None,
 'CompletedAt': None,
 'Status': 'Queued',
 'Duration': None,
 'FailureReason': None,
 'Name': 'Example name'}

## Wait for the solve operation to complete
The operation's `await` endpoint is designed to respond as soon as the operation has finished (within 1 minute). You can continue sending `await` requests until the status of the returned operation is no longer in progress, in case the operation takes longer than 1 minute to complete.

In [5]:
operationId = solveOperation["Id"]
while solveOperation["Status"] in ["Queued", "InProgress"]:
    solveOperationResponse = requests.get(f"{base_url}/v1/operations/{operationId}/await", headers=headers)
    solveOperation = solveOperationResponse.json()
    
solveOperation

{'Artifacts': [{'Id': '90cdebaf-1ad0-42b2-b43e-3dcced22b335',
   'Filename': 'Example name.3fs',
   'Type': 'text/plain',
   'DownloadUrl': 'https://model.oxfordeconomics.com/api/v1/operations/23613/artifact/90cdebaf-1ad0-42b2-b43e-3dcced22b335'},
  {'Id': '9765f37c-2cc7-4f3d-8960-616ebe95afbb',
   'Filename': 'Example name.run',
   'Type': 'text/plain',
   'DownloadUrl': 'https://model.oxfordeconomics.com/api/v1/operations/23613/artifact/9765f37c-2cc7-4f3d-8960-616ebe95afbb'},
  {'Id': 'a9515071-b9f5-433e-9258-a4709bb3aacf',
   'Filename': 'Example name.out',
   'Type': 'text/plain',
   'DownloadUrl': 'https://model.oxfordeconomics.com/api/v1/operations/23613/artifact/a9515071-b9f5-433e-9258-a4709bb3aacf'},
  {'Id': 'b96724b8-faa5-4f78-9dc7-52ead8e93447',
   'Filename': 'Example name-solver-info.json',
   'Type': 'application/json',
   'DownloadUrl': 'https://model.oxfordeconomics.com/api/v1/operations/23613/artifact/b96724b8-faa5-4f78-9dc7-52ead8e93447'}],
 'Resources': [{'Id': '2198

## Verify that the solution succeeded
Model solutions can fail for various reasons (e.g. inapplicable model commands, failure to converge). Check that the operation's `Status` is `"Succeeded"` before proceeding to use the new forecast. If it failed, `FailureReason` may help you understand what went wrong.

In [6]:
if (solveOperation["Status"] != "Succeeded"):
    print("Model solution failed\n\n" + solveOperation["FailureReason"])
else:
    print(f"Model solution succeeded in {solveOperation['Duration']}ms")


Model solution succeeded in 78795ms


## Inspect the newly-solved forecast

There are many ways to use the new forecast once it's ready (e.g. exporting data, using it another model solution). In this example, we're simply fetching some properties for it from the `/resources` endpoint.

In [7]:
outputForecastResource = requests.get(f"{base_url}/v1/resources/" + solveRequest["OutputForecast"], headers=headers)
outputForecast = outputForecastResource.json()
outputForecast

{'Type': 'Forecast',
 'EconomicDomain': 'MACRO',
 'ForecastUrl': '/v1/forecasts/=99380406-d06f-43ea-acc5-528e79caf0a8',
 'Versions': [{'Name': 'new-forecast',
   'EconomicDomain': 'MACRO',
   'Range': {'From': '1980Q1', 'To': '2050Q4'},
   'Id': '89583b9f-a58e-463a-9282-64bc3464e8fe',
   'Version': 0,
   'CreatedAt': '2023-10-16T10:19:59.1693614+00:00'}],
 'ResourceType': 'Forecast',
 'Id': '99380406-d06f-43ea-acc5-528e79caf0a8',
 'Name': 'new-forecast',
 'Path': '/tim-stott/path/to/new-forecast',
 'Archiving': None,
 'Product': {'TypeCode': 'OEF', 'Code': 'GBLMACRLM25_ONLINE'}}

## Inspecting artifacts
Besides an output forecast, each model solution also generates a few other files known as "artifacts". In some cases, particularly if there is a problem with the solution, it may be useful to examine these files. In this example, we get the contents of the solution's `.out` file, which contains detailed information about the model solution process.

In [11]:
outFileArtifact = list(filter(lambda artifact: (artifact["Filename"].endswith('.out')), solveOperation["Artifacts"]))[0]
outFileResponse = requests.get(outFileArtifact["DownloadUrl"], headers=headers)
outFileContents = outFileResponse.text
print(outFileContents)

 Genmod Version 2013.11.01 
                                                                                             16/10/2023  10.19.02
 11,12,20232-20274,,4,TQSF                                                       
  Input file 11     ï»¿OEGEM                                                                            11/10/2023  09.20.40
 Output file 12 computed from 2023  2 to 2027  4 and stored from 1980  1 to 2050  4    Number of periods per year =  4
 INTRAN = 1     INEQU = 1     ) non-zero if the transformation/equation/second residual/fix
  INADD = 1     INFIX = 1     ) arrays are to be read from the input file
 An array of size     231 has been used by the input V,R and F line data - maximum size allowed is 20000000
 ** was the maximum lag used in the trial solution -at least 4 lagged values are allocated to all variables and residuals
 An array of size  1342771 has been used for the variables and residuals - maximum size allowed is 20000000
 Data input lines have been

## Applying changes without solving
To apply changes inline or from a file without immediately solving the model, simply use the `/operations/apply` endpoint and omit the `SolutionRange` and `Groups` properties. Other options remain the same, except that you must provide `Commands` or `CommandsFile` (for `solve`, it is permissible to omit both).

In [ ]:
# The solution request body expects JSON of the form below.
# Optional properties are commented out, with their default values shown

applyRequest = {
    # Path to the input forecast
    "InputForecast": sourceForecast["Path"],

    ## Inline model commands to be applied
    ## Cannot be used with "CommandsFile"
    # "Commands": "MULTIPLY C:US 2023Q2 1.01, 1.01, 1.02\nFIX C:US 2023Q2:2028Q4",

    ## Path to file containing model commands or valid import data to be applied
    ## Cannot be used with "Commands"
    # "CommandsFile": "/me/path/to/commands.3fs",
    # "CommandsFile": "/me/path/to/data.xlsx",

    ## Optional name for the apply operation. This name will appear in API requests
    ## for this operation and in the 'History' panel of the Run Model user interface. It's also 
    ## used to name the other artifacts generated during the operation (besides the output forecast, see below)
    # "OperationName": "Example name",

    # Path at which to save the newly-solved forecast
    "OutputForecast": "/me/path/to/new-forecast"
    
}
applyOperationResponse = requests.post(f"{base_url}/v1/operations/apply", headers=headers, json=applyRequest)
applyOperation = applyOperationResponse.json()
applyOperation